<img src="../assets/logos/Logo_layout.png" width=500></img>


# Second Deploy: Analytics
Your first model just randomly places ships of size 2. Since you can make ships of many different shapes we would like to find what is the best shape. You also need to change the shape of your ships since other teams shooting models will learn the patterns of your layout. In this model, each game your model will choose one of two different layout strategies and you will then use [Amazon Athena](https://aws.amazon.com/athena/) to see the results. You can then repeate the experiment with new ship sizes and shapes to see if you can find better strategies. 

## Code 
The code for your second model is in /containers/2vs3. In addition to randomly selecting a layout strategy it will also put a tag in the session attribute that we will use in athena to compare different games. Next, run the test script localy to confirm the scripts works. 

## Configure
Before you can deploy your second model you need to configure the deployment. As described in the [Endpoint Notebook](../Endpoint_Reference.ipynb), you will edit to the deployment configuration in [/deploy-config.json](/edit/GameDayRepo/deploy-config.json). you need to edit the following field to the following values

|parameter field|Value|
|---|---|
|trainsourcefile|`layout-2vs3.tar.gz`|
|hostsourcefile|`layout-2vs3.tar.gz`|



## Deploy
1. follow instructions from endpoint reference to commit your code and changes and then push your changes to the remote branch
2. tell opperations to deploy your changes

## Amazon Athena
You will use Amazon Athena to monitor the performance of your model. Athena allows you to run SQL queries against data in s3, like the game archive data. You can see a example query by running the following command:

In [ ]:
!../../bin/glue_info.sh

1. Go to the [Athena Console](https://console.aws.amazon.com/athena/home?region=us-east-1#query)
1. you are going to use the following query but you need to first get some values. You need the name of the Amazon Glue catalog that holds the game data and the s3 bucket where that data is stored. These values are given by the above command. Replace <Game-data-catalog> and <Game-data-bucket> with those values. so for example the line `FROM "<Game-data-catalog>"."<Game-data-bucket>"` will look something like `FROM "game-catalog-adafafdad"."game-archive-adfasdfasdf"`
1. Next replace the three occurances of <Team-Name> with your team name. For example, the line `WHEN winner = '<Team-Name>' THEN 1.0` becomes `WHEN winner = 'my-team' THEN 1.0`

```sql
SELECT type,
         sum(won)/(sum(won)+sum(lost)) as won,
         sum(lost)/(sum(won)+sum(lost)) as lost
FROM 
    (SELECT json_extract(session,'$.type') AS type,
        CASE
            WHEN winner = '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS won,
        CASE
            WHEN winner != '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS lost
    FROM 
        (SELECT teama.session AS session,
         teama.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>"
        UNION
        SELECT teamb.session AS session,
         teamb.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>" )
        WHERE team = '<Team-Name>' )
    GROUP BY  type
```

## Try again
1. program some new ship sizes or shapes to do another round of experiments with
2. deploy it out and rerun your athena query

## Next Step
1. go to [third deploy](./Third_Deploy.ipynb) to try the next strategy